In [8]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt


In [10]:
n_qubits = 3

qubits = cirq.GridQubit.rect(1, n_qubits)
g=1
J=1.21
observable = [-float(0.5*g)*cirq.Z.on(q) for q in qubits] 
for q in range(len(qubits)):
    observable.append(-float(0.5*J)*cirq.X.on(qubits[q])*cirq.X.on(qubits[(q+1)%len(qubits)]))
circuit = []

parametrized_unitary = [cirq.rz, cirq.rx, cirq.rz]

indexed_cnots = {}
cnots_index = {}
count = 0
for control in range(n_qubits):
    for target in range(n_qubits):
        if control != target:
            indexed_cnots[str(count)] = [control, target]
            cnots_index[str([control,target])] = count
            count += 1
number_of_cnots = len(indexed_cnots)

def append_to_circuit(ind, circuit, params, new_index=False):
    if ind < number_of_cnots:
        control, target = indexed_cnots[str(ind)]
        circuit.append(cirq.CNOT.on(qubits[control], qubits[target]))
        return circuit, params
    else:
        qubit = qubits[(ind-number_of_cnots)%n_qubits]
        for par, gate in zip(range(3),parametrized_unitary):
            new_param = "th_"+str(len(params))
            params.append(new_param)
            circuit.append(gate(sympy.Symbol(new_param)).on(qubit))
        return circuit, params
    
def give_circuit(lista):
    circuit, symbols = [], []
    for k in lista:
        circuit, symbols = append_to_circuit(k,circuit,symbols)
    circuit = cirq.Circuit(circuit)
    return circuit, symbols

In [15]:
circuit,symbols = give_circuit([number_of_cnots,number_of_cnots+1,number_of_cnots+2])

In [16]:
circuit

(0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───

(0, 1): ───Rz(th_3)───Rx(th_4)───Rz(th_5)───

(0, 2): ───Rz(th_6)───Rx(th_7)───Rz(th_8)───

In [17]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.00001)))(
circuit_input,
symbol_names=symbols,
operators=tfq.convert_to_tensor([observable]),
initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse')

tfqcircuit = tfq.convert_to_tensor([circuit])
qoutput = tf.ones((1, 1))*-2.*n_qubits

In [18]:
model.get_weights()

[array([-0.04280701, -0.02854302, -0.06219545, -0.00624329,  0.11626633,
        -0.04399979, -0.03742867,  0.01061319, -0.0083705 ], dtype=float32)]

In [35]:
for k in tqdm(range(120)):
    model.set_weights([tf.random.uniform((len(symbols),))])
    model.fit(x=tfqcircuit, y=qoutput,validation_steps=None,
              steps_per_epoch=1, batch_size=1, epochs=20, verbose=1, 
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

  0%|          | 0/120 [00:00<?, ?it/s]

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 0s 461ms/sample - loss: 33.1257
Epoch 2/20
1/1 [==============================] - 1s 613ms/sample - loss: 33.1258
Epoch 3/20
1/1 [==============================] - 1s 506ms/sample - loss: 33.1259
Epoch 4/20
1/1 [==============================] - 0s 465ms/sample - loss: 33.1261
Epoch 5/20
1/1 [==============================] - 1s 1s/sample - loss: 33.1263
Epoch 6/20
1/1 [==============================] - 1s 987ms/sample - loss: 33.1265


  1%|          | 1/120 [00:04<08:04,  4.07s/it]

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 0s 460ms/sample - loss: 33.1510
Epoch 2/20
1/1 [==============================] - 0s 457ms/sample - loss: 33.1434
Epoch 3/20
1/1 [==============================] - 0s 457ms/sample - loss: 33.1324
Epoch 4/20
1/1 [==============================] - 1s 531ms/sample - loss: 33.1184
Epoch 5/20
1/1 [==============================] - 1s 952ms/sample - loss: 33.1019
Epoch 6/20
1/1 [==============================] - 1s 1s/sample - loss: 33.0832
Epoch 7/20
1/1 [==============================] - 2s 2s/sample - loss: 33.0627
Epoch 8/20
1/1 [==============================] - 1s 1s/sample - loss: 33.0405
Epoch 9/20
1/1 [==============================] - 1s 646ms/sample - loss: 33.0170
Epoch 10/20
1/1 [==============================] - 1s 630ms/sample - loss: 32.9924
Epoch 11/20
1/1 [==============================] - 1s 527ms/sample - loss: 32.9669
Epoch 12/20
1/1 [==============================] - 1s 688ms/sample - loss: 32.9407
Epo

  2%|▏         | 2/120 [00:18<13:58,  7.11s/it]

...

 98%|█████████▊| 118/120 [22:41<00:20, 10.24s/it]

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 1s 628ms/sample - loss: 33.4141
Epoch 2/20
1/1 [==============================] - 1s 550ms/sample - loss: 33.4119
Epoch 3/20
1/1 [==============================] - 0s 437ms/sample - loss: 33.4086
Epoch 4/20
1/1 [==============================] - 1s 514ms/sample - loss: 33.4042
Epoch 5/20
1/1 [==============================] - 1s 505ms/sample - loss: 33.3988
Epoch 6/20
1/1 [==============================] - 1s 644ms/sample - loss: 33.3925
Epoch 7/20
1/1 [==============================] - 1s 586ms/sample - loss: 33.3853
Epoch 8/20
1/1 [==============================] - 0s 471ms/sample - loss: 33.3774
Epoch 9/20
1/1 [==============================] - 1s 627ms/sample - loss: 33.3688
Epoch 10/20
1/1 [==============================] - 1s 515ms/sample - loss: 33.3596
Epoch 11/20
1/1 [==============================] - 1s 569ms/sample - loss: 33.3498
Epoch 12/20
1/1 [==============================] - 0s 486ms/sample - loss: 33

 99%|█████████▉| 119/120 [22:52<00:10, 10.44s/it]

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 0s 471ms/sample - loss: 33.7590
Epoch 2/20
1/1 [==============================] - 0s 492ms/sample - loss: 33.7469
Epoch 3/20
1/1 [==============================] - 0s 471ms/sample - loss: 33.7330
Epoch 4/20
1/1 [==============================] - 0s 446ms/sample - loss: 33.7175
Epoch 5/20
1/1 [==============================] - 1s 616ms/sample - loss: 33.7008
Epoch 6/20
1/1 [==============================] - 1s 809ms/sample - loss: 33.6830
Epoch 7/20
1/1 [==============================] - 0s 498ms/sample - loss: 33.6644
Epoch 8/20
1/1 [==============================] - 1s 569ms/sample - loss: 33.6453
Epoch 9/20
1/1 [==============================] - 1s 700ms/sample - loss: 33.6257
Epoch 10/20
1/1 [==============================] - 0s 462ms/sample - loss: 33.6058
Epoch 11/20
1/1 [==============================] - 1s 654ms/sample - loss: 33.5859
Epoch 12/20
1/1 [==============================] - 1s 603ms/sample - loss: 33

100%|██████████| 120/120 [23:03<00:00, 11.53s/it]


In [ ]:
np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))

In [ ]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(
circuit_input,
symbol_names=symbols,
operators=tfq.convert_to_tensor([observable]),
initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse')
tfqcircuit = tfq.convert_to_tensor([circuit])
qoutput = tf.ones((1, 1))*-2.*n_qubits
model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=250, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

In [ ]:
np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))